# Weather API Client Demos
Use this notebook to verify the local weather API wrappers for Tomorrow.io, Open-Meteo, Visual Crossing, NOAA CDO, WeatherAPI.com, OpenWeather, and Weatherbit.


In [1]:
import json
from pathlib import Path
import sys
PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
CONFIG_PATH = Path('../weather_config.json')
CONFIG = json.loads(CONFIG_PATH.read_text())
LOCATIONS = CONFIG.get('locations', {})
LOCATION_ITEMS = []
for name, coords in LOCATIONS.items():
    try:
        lat = float(coords['lat'])
        lon = float(coords['lon'])
    except (KeyError, TypeError, ValueError) as exc:
        raise ValueError(f"Invalid coordinates for location '{name}'. Provide numeric 'lat' and 'lon'.") from exc
    LOCATION_ITEMS.append((name, lat, lon))
if not LOCATION_ITEMS:
    raise ValueError("Define at least one location under 'locations' in weather_config.json.")
def iter_locations():
    for item in LOCATION_ITEMS:
        yield item
LOCATION_ITEMS
from clients.noaa_access_client import NoaaIsdClient, NoaaLcdClient
from clients.meteostat_client import MeteostatClient
from clients.nasa_power_client import NasaPowerClient
from clients.iem_asos_client import IemAsosClient
from clients.copernicus_cds_client import CopernicusCdsClient


/Users/chikro/Downloads/DL Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Tomorrow.io client
The client reads API keys and defaults from `weather_config.json`. Update the config file before running the calls below.

In [2]:
from clients.tomorrow_io_client import TomorrowIOClient

client = TomorrowIOClient(config_path="../weather_config.json")
client.base_url


'https://api.tomorrow.io/v4'

## Forecast example
Requests the next 6 hours of weather data for Boston, MA. Adjust the location or fields for your use case.

In [3]:
forecast_results = {}
for name, lat, lon in iter_locations():
    request = {
        "location": f"{lat},{lon}",
        "timeframe_hours": 6,
        "fields": ["temperature", "humidity", "windSpeed"],
        "timesteps": ["1h"],
        "units": "metric",
        "timezone": "UTC",
    }
    forecast_results[name] = {
        "single": client.get_forecast(**request),
        "batch": client.get_forecast_batch([request])[0],
    }
forecast_results


ApiError: Tomorrow.io error 429: {"code":429001,"type":"Too Many Calls","message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume."}

## Historical example
Fetches the last 24 hours leading up to the current time. `start` and `end` must be in ISO-8601.

In [ ]:
import datetime as dt

historical_results = {}
end_time = dt.datetime.now(dt.timezone.utc).replace(microsecond=0)
start_time = end_time - dt.timedelta(hours=24)

for name, lat, lon in iter_locations():
    request = {
        "location": f"{lat},{lon}",
        "start_time": start_time,
        "end_time": end_time,
        "fields": ["temperature", "humidity", "windSpeed"],
        "timesteps": ["1h"],
        "units": "metric",
        "timezone": "UTC",
    }
    historical_results[name] = {
        "single": client.get_historical(**request),
        "batch": client.get_historical_batch([request])[0],
    }
historical_results


## Next steps
Explore the payloads, convert them to pandas DataFrames, or export to other formats as needed.

## Open-Meteo client
Open-Meteo does not require an API key. Defaults (timezone, hourly variables, units) come from `weather_config.json`.

In [ ]:
from clients.open_meteo_client import OpenMeteoClient

open_meteo = OpenMeteoClient(config_path="../weather_config.json")
open_meteo.forecast_url


## Open-Meteo forecast example
Requests the upcoming 2 days of data for Boston, MA. Adjust location, variables, or duration to suit your needs.

In [ ]:
open_meteo_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
        "daily": ["temperature_2m_max", "temperature_2m_min"],
        "forecast_days": 2,
    }
    open_meteo_forecasts[name] = {
        "single": open_meteo.get_forecast(**request),
        "batch": open_meteo.get_forecast_batch([request])[0],
    }
open_meteo_forecasts


## Open-Meteo historical example
Fetches the previous 5 days of archive data for the same location. Dates must be provided in ISO format.

In [ ]:
import datetime as dt
historical_end = dt.date.today()
historical_start = historical_end - dt.timedelta(days=5)

open_meteo_historical = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "start_date": historical_start,
        "end_date": historical_end,
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
        "daily": ["temperature_2m_max", "temperature_2m_min"],
    }
    open_meteo_historical[name] = {
        "single": open_meteo.get_historical(**request),
        "batch": open_meteo.get_historical_batch([request])[0],
    }
open_meteo_historical


## Visual Crossing client
Visual Crossing uses the Timeline Weather API and requires an API key provided in `weather_config.json`.

In [ ]:
from clients.visual_crossing_client import VisualCrossingClient

visual_crossing = VisualCrossingClient(config_path="../weather_config.json")
visual_crossing.base_url


## Visual Crossing forecast example
Retrieves upcoming conditions for the configured coordinates, including current, daily, and hourly data segments.

In [ ]:
visual_crossing_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": f"{lat},{lon}",
        "include": ["current", "days", "hours"],
        "unit_group": "metric",
    }
    visual_crossing_forecasts[name] = {
        "single": visual_crossing.get_forecast(**request),
        "batch": visual_crossing.get_forecast_batch([request])[0],
    }
visual_crossing_forecasts


## Visual Crossing historical example
Queries the previous 7 days using the timeline endpoint with explicit start and end dates.

In [ ]:
import datetime as dt

vc_end = dt.date.today()
vc_start = vc_end - dt.timedelta(days=7)

visual_crossing_historical = {}
for name, lat, lon in iter_locations():
    request = {
        "location": f"{lat},{lon}",
        "start": vc_start,
        "end": vc_end,
        "include": ["days", "hours"],
    }
    visual_crossing_historical[name] = {
        "single": visual_crossing.get_historical(**request),
        "batch": visual_crossing.get_historical_batch([request])[0],
    }
visual_crossing_historical


## NOAA ISD client
Integrated Surface Database (global-hourly) via the Access Data Service.


In [ ]:
noaa_isd = NoaaIsdClient(config_path="../weather_config.json")
noaa_isd.base_url, noaa_isd.dataset


('https://www.ncei.noaa.gov/access/services/data/v1', 'global-hourly')

## NOAA ISD observation example
Fetch the past 24 hours of hourly observations for each configured location.


In [ ]:
import datetime as dt

DEMO_NOAA_ISD_START = dt.datetime(2024, 3, 1, tzinfo=dt.timezone.utc)
DEMO_NOAA_ISD_END = DEMO_NOAA_ISD_START + dt.timedelta(days=1)

noaa_isd_results = {}
for name, *_coords in iter_locations():
    station_id = LOCATIONS.get(name, {}).get('noaaIsdStation')
    if not station_id:
        print(f"{name}: missing 'noaaIsdStation'; skipping.")
        continue
    payload = noaa_isd.get_observations(
        station_id=station_id,
        start_time=DEMO_NOAA_ISD_START,
        end_time=DEMO_NOAA_ISD_END,
    )
    noaa_isd_results[name] = payload
    print(f"{name}: {len(payload)} ISD rows for {DEMO_NOAA_ISD_START.date()}")

sample_isd_city, sample_isd_payload = next(iter(noaa_isd_results.items()))
pd.DataFrame(sample_isd_payload)[['DATE', 'TMP', 'DEW', 'WND']].head()


new_york_ny: 26 ISD rows for 2024-03-01
austin_tx: 47 ISD rows for 2024-03-01
chicago_midway_il: 30 ISD rows for 2024-03-01
los_angeles_ca: 41 ISD rows for 2024-03-01
miami_fl: 36 ISD rows for 2024-03-01


,DATE,TMP,DEW,WND
0,2024-03-01T00:51:00,"+0033,5","-0100,5","290,5,N,0021,5"
1,2024-03-01T01:51:00,"+0022,5","-0117,5","290,5,N,0026,5"
2,2024-03-01T02:51:00,"+0017,5","-0117,5","999,9,V,0015,5"
3,2024-03-01T03:51:00,"+0017,5","-0117,5","300,5,N,0036,5"
4,2024-03-01T04:51:00,"+0011,5","-0117,5","999,9,V,0021,5"


## NOAA LCD observation example
Hourly Local Climatological Data via the same access service.


In [ ]:
import datetime as dt

DEMO_NOAA_LCD_START = dt.datetime(2024, 3, 1, tzinfo=dt.timezone.utc)
DEMO_NOAA_LCD_END = DEMO_NOAA_LCD_START + dt.timedelta(days=1)

noaa_lcd = NoaaLcdClient(config_path="../weather_config.json")
noaa_lcd_results = {}
for name, *_coords in iter_locations():
    station_id = LOCATIONS.get(name, {}).get('noaaLcdStation')
    if not station_id:
        print(f"{name}: missing 'noaaLcdStation'; skipping.")
        continue
    payload = noaa_lcd.get_observations(
        station_id=station_id,
        start_time=DEMO_NOAA_LCD_START,
        end_time=DEMO_NOAA_LCD_END,
    )
    noaa_lcd_results[name] = payload
    print(f"{name}: {len(payload)} LCD rows for {DEMO_NOAA_LCD_START.date()}")


new_york_ny: 25 LCD rows for 2024-03-01
austin_tx: 41 LCD rows for 2024-03-01
chicago_midway_il: 29 LCD rows for 2024-03-01
los_angeles_ca: 40 LCD rows for 2024-03-01
miami_fl: 34 LCD rows for 2024-03-01


In [ ]:
sample_lcd_city, sample_lcd_payload = next(iter(noaa_lcd_results.items()))
pd.DataFrame(sample_lcd_payload)[["DATE", "HourlyDryBulbTemperature", "HourlyDewPointTemperature"]].head()


,DATE,HourlyDryBulbTemperature,HourlyDewPointTemperature
0,2024-03-01T00:51:00,0.0,-11.1
1,2024-03-01T01:51:00,-0.6,-11.7
2,2024-03-01T02:51:00,-0.6,-11.1
3,2024-03-01T03:51:00,-1.1,-11.7
4,2024-03-01T04:51:00,-1.1,-11.1


## Meteostat client
Hourly observations via the Meteostat Python library (Point API).


In [ ]:
meteostat_client = MeteostatClient(config_path="../weather_config.json")


In [ ]:
import datetime as dt
METEOSTAT_DEMO_START = dt.datetime(2024, 3, 1)
METEOSTAT_DEMO_END = METEOSTAT_DEMO_START + dt.timedelta(days=1)
meteostat_results = {}
for name, lat, lon in iter_locations():
    payload = meteostat_client.get_hourly(
        location=(lat, lon),
        start_time=METEOSTAT_DEMO_START,
        end_time=METEOSTAT_DEMO_END,
    )
    meteostat_results[name] = payload
    print(f"{name}: {len(payload)} Meteostat rows for {METEOSTAT_DEMO_START.date()}")
sample_meteostat_city, sample_meteostat_payload = next(iter(meteostat_results.items()))
pd.DataFrame(sample_meteostat_payload)[['timestamp', 'temp', 'dwpt']].head()


## NASA POWER client
NASA POWER hourly API via REST.


In [ ]:
nasa_power = NasaPowerClient(config_path="../weather_config.json")


In [ ]:
import datetime as dt

NASA_DEMO_START = dt.datetime(2024, 3, 1)
NASA_DEMO_END = NASA_DEMO_START + dt.timedelta(days=1)

nasa_power_results = {}
for name, lat, lon in iter_locations():
    payload = nasa_power.get_hourly(
        location=(lat, lon),
        start_time=NASA_DEMO_START,
        end_time=NASA_DEMO_END,
    )
    nasa_power_results[name] = payload
    count = 0
    props = payload.get('properties', {}) if isinstance(payload, dict) else {}
    param_block = props.get('parameter', {})
    if param_block:
        first_param = next(iter(param_block.values()))
        count = len(first_param)
    print(f"{name}: {count} NASA POWER rows for {NASA_DEMO_START.date()}")

sample_nasa_city, sample_nasa_payload = next(iter(nasa_power_results.items()))
parameters = sample_nasa_payload.get('properties', {}).get('parameter', {})
rows = {}
for param, series in parameters.items():
    for stamp, value in series.items():
        rows.setdefault(stamp, {})[param] = value
records = []
for stamp, values in rows.items():
    ts = dt.datetime.strptime(stamp, '%Y%m%d%H')
    values = dict(values)
    values['timestamp'] = ts
    records.append(values)
pd.DataFrame(records)[['timestamp', 'T2M', 'RH2M', 'WS10M']].head()


new_york_ny: 48 NASA POWER rows for 2024-03-01
austin_tx: 48 NASA POWER rows for 2024-03-01
chicago_midway_il: 48 NASA POWER rows for 2024-03-01
los_angeles_ca: 48 NASA POWER rows for 2024-03-01
miami_fl: 48 NASA POWER rows for 2024-03-01


,timestamp,T2M,RH2M,WS10M
0,2024-03-01 00:00:00,-2.25,82.05,3.62
1,2024-03-01 01:00:00,-2.98,85.25,3.30
2,2024-03-01 02:00:00,-3.44,87.24,3.04
3,2024-03-01 03:00:00,-3.62,87.88,2.84
4,2024-03-01 04:00:00,-3.78,89.09,2.62


## Copernicus ERA5 datasets
Four CDS-backed providers are configured: ERA5 single levels, ERA5-Land, ERA5 pressure levels, and ERA5-Land time-series. Each provider uses its own bounding box metadata in `weather_config.json`. The cell below loops through all four providers and fetches a one-day sample for every configured location.

In [5]:
from IPython.display import display
import datetime as dt

COPERNICUS_DEMOS = [
    {
        "label": "ERA5 single levels",
        "provider": "copernicus_era5_single",
        "area_attr": "copernicusEra5Area",
        "date": dt.date(2024, 1, 1),
    },
    {
        "label": "ERA5-Land",
        "provider": "copernicus_era5_land",
        "area_attr": "copernicusEra5LandArea",
        "date": dt.date(2024, 1, 1),
    },
    {
        "label": "ERA5 pressure levels",
        "provider": "copernicus_era5_pressure",
        "area_attr": "copernicusEra5Area",
        "date": dt.date(2024, 1, 1),
    },
    {
        "label": "ERA5-Land time-series",
        "provider": "copernicus_era5_land_timeseries",
        "area_attr": None,
        "date": dt.date(2024, 1, 1),
    },
]


def _run_copernicus_demo(cfg):
    client = CopernicusCdsClient(config_path="../weather_config.json", provider=cfg["provider"])
    results = {}
    for name, lat, lon in iter_locations():
        extras = LOCATIONS.get(name, {})
        area_attr = cfg.get("area_attr")
        area = extras.get(area_attr) if area_attr else None
        if area_attr and not area:
            print(f"{cfg['label']} -> {name}: missing {area_attr}; skipping.")
            continue
        payload = client.get_dataset(
            area=area,
            start_date=cfg["date"],
            end_date=cfg["date"],
            latitude=lat,
            longitude=lon,
        )
        results[name] = payload
        length = 0 if payload is None else len(payload)
        print(f"{cfg['label']} -> {name}: {length} rows on {cfg['date']}")
    return results


def _preview_dataframe(df):
    cols = [col for col in ("timestamp", "pressure_level", "t2m", "tp", "temperature", "u10", "v10") if col in df.columns]
    if not cols:
        cols = list(df.columns[:5])
    return df[cols].head()

copernicus_results = {}
for cfg in COPERNICUS_DEMOS:
    print(f"=== {cfg['label']} ===")
    copernicus_results[cfg["provider"]] = _run_copernicus_demo(cfg)

for provider_key, results in copernicus_results.items():
    sample_item = next(((city, payload) for city, payload in results.items() if payload is not None and not payload.empty), None)
    if sample_item:
        city, payload = sample_item
        print(f"Sample preview for {provider_key} ({city})")
        display(_preview_dataframe(payload))
    else:
        print(f"No sample data to preview for {provider_key}.")


=== ERA5 single levels ===


ERA5 single levels -> new_york_ny: 24 rows on 2024-01-01


Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 1 of 500
Retrying in 120 seconds


KeyboardInterrupt: 

## IEM ASOS client
Iowa Environmental Mesonet 1-minute ASOS data.


In [ ]:
iem_asos = IemAsosClient(config_path="../weather_config.json")


In [ ]:
import datetime as dt

IEM_DEMO_START = dt.datetime(2025, 1, 1)
IEM_DEMO_END = IEM_DEMO_START + dt.timedelta(days=1)

iem_results = {}
for name, *_coords in iter_locations():
    station = LOCATIONS.get(name, {}).get('iemStation')
    network = LOCATIONS.get(name, {}).get('iemNetwork')
    if not station or not network:
        print(f"{name}: missing IEM metadata; skipping.")
        continue
    payload = iem_asos.get_observations(
        station=station,
        network=network,
        start_time=IEM_DEMO_START,
        end_time=IEM_DEMO_END,
    )
    iem_results[name] = payload
    length = 0 if payload is None else len(payload)
    print(f"{name}: {length} IEM rows for {IEM_DEMO_START.date()}")

sample_item = next(((city, payload) for city, payload in iem_results.items() if payload is not None and not payload.empty), None)
if sample_item:
    sample_city, sample_payload = sample_item
    pd.DataFrame(sample_payload)[['timestamp', 'tmpf', 'dwpf', 'sknt']].head()
else:
    print('No IEM sample data available.')


new_york_ny: 1130 IEM rows for 2025-01-01
austin_tx: 937 IEM rows for 2025-01-01
chicago_midway_il: 1249 IEM rows for 2025-01-01
los_angeles_ca: 960 IEM rows for 2025-01-01
miami_fl: 1270 IEM rows for 2025-01-01


## WeatherAPI.com client
WeatherAPI.com uses API key authentication via query parameters; the key lives in `weather_config.json`.

In [ ]:
from clients.weatherapi_com_client import WeatherApiClient

weatherapi = WeatherApiClient(config_path="../weather_config.json")
weatherapi.base_url


## WeatherAPI.com forecast example
Requests a 3-day forecast (includes hourly segments) for the configured coordinates.

In [ ]:
weatherapi_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "days": 3,
        "aqi": 'yes',
        "alerts": 'yes',
    }
    weatherapi_forecasts[name] = {
        "single": weatherapi.get_forecast(**request),
        "batch": weatherapi.get_forecast_batch([request])[0],
    }
weatherapi_forecasts


## WeatherAPI.com historical example
Fetches hourly history for a single day (WeatherAPI supports one date per call on standard plans).

In [ ]:
import datetime as dt
history_date = dt.date.today() - dt.timedelta(days=1)

weatherapi_history = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "date": history_date,
        "aqi": 'yes',
    }
    weatherapi_history[name] = {
        "single": weatherapi.get_historical(**request),
        "batch": weatherapi.get_historical_batch([request])[0],
    }
weatherapi_history


## OpenWeather client
OpenWeather current and historical APIs both rely on latitude/longitude plus an API key stored in `weather_config.json`.


In [ ]:
import importlib
from clients import openweather_client as _openweather_module
importlib.reload(_openweather_module)
from clients.openweather_client import OpenWeatherClient

openweather = OpenWeatherClient(config_path="../weather_config.json")
openweather.current_url, openweather.history_url


## OpenWeather current example
Fetches the latest observations for the configured coordinates.


In [ ]:
current_results = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "units": 'metric',
        "language": 'en',
    }
    current_results[name] = {
        "single": openweather.get_current(**request),
        "batch": openweather.get_current_batch([request])[0],
    }
current_results


## OpenWeather historical example
Requests sub-hourly history for the last 24 hours (data is returned in hourly buckets).


In [ ]:
import datetime as dt

ow_end = dt.datetime.now(dt.timezone.utc)
ow_start = ow_end - dt.timedelta(hours=24)

openweather_history = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "start_time": ow_start,
        "end_time": ow_end,
        "units": 'metric',
        "interval_type": 'hour',
    }
    openweather_history[name] = {
        "single": openweather.get_historical(**request),
        "batch": openweather.get_historical_batch([request])[0],
    }
openweather_history


## Weatherbit client
Weatherbit exposes hourly forecast data and sub-hourly historical archives; the API key lives in `weather_config.json`.


In [ ]:
from clients.weatherbit_client import WeatherbitClient

weatherbit = WeatherbitClient(config_path="../weather_config.json")
weatherbit.forecast_url


## Weatherbit forecast example
Requests the next 48 hours of hourly forecast data for the configured coordinates.


In [ ]:
weatherbit_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "hours": 48,
        "units": 'M',
    }
    weatherbit_forecasts[name] = {
        "single": weatherbit.get_forecast(**request),
        "batch": weatherbit.get_forecast_batch([request])[0],
    }
weatherbit_forecasts


## Weatherbit historical example
Fetches the previous 24 hours using the sub-hourly historical endpoint (UTC timestamps required).


In [ ]:
import datetime as dt

wb_end = dt.datetime.now(dt.timezone.utc)
wb_start = wb_end - dt.timedelta(hours=24)

weatherbit_history = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "start_time": wb_start,
        "end_time": wb_end,
        "units": 'M',
        "tz": 'UTC',
    }
    weatherbit_history[name] = {
        "single": weatherbit.get_historical(**request),
        "batch": weatherbit.get_historical_batch([request])[0],
    }
weatherbit_history
